In [25]:
import mwu_measures
import pandas as pd
import numpy as np
import mwu_measures.processing_corpus
from mwu_measures.corpus_helper import Fetcher

mwu_examples = pd.read_csv('MultiwordExpression_Concreteness_Ratings.csv')
mwu_examples['length'] = mwu_examples['Expression'].apply(lambda x: len(x.split()))
mwu_examples = mwu_examples.loc[(mwu_examples['length'] == 2) | (mwu_examples['length'] == 3)]
mwu_examples['Expression'] = mwu_examples['Expression'].apply(lambda x: x.lower())
this_corpus = mwu_measures.processing_corpus.process_corpus('bnc', 'small_corpus.txt', chunk_size=10000000, verbose=True)
ngram_selection = mwu_examples['Expression'].tolist()
bigrams = [ngram for ngram in ngram_selection if len(ngram.split()) == 2]
# ngram_selection = [ngram.split() for ngram in ngram_selection]
# ngram_chunks = np.array_split(ngram_selection, 100)

90357 lines processed
178289 lines processed
287795 lines processed
301046 lines processed
we bring the boom


In [26]:
bigrams

['.22 caliber',
 '.38 caliber',
 '1 kgs',
 '1 samuel',
 '1 timothy',
 '10-undecenoic acid',
 '18-karat gold',
 '1st class',
 '1st-class mail',
 '20/20 vision',
 '22-karat gold',
 '30 minutes',
 '365 days',
 '3d printer',
 '3d scanner',
 '4-3 suspension',
 '4-h club',
 '8 ball',
 '800 number',
 'a battery',
 'a bit',
 'a blowhard',
 'a bunch',
 'a capella',
 'a cappella',
 'a couple',
 'à deux',
 'a doddle',
 'a drive',
 'a few',
 'a fortiori',
 'a freak-out',
 'a further',
 'a getaway',
 'a go',
 'a goner',
 'a good',
 'a goof-off',
 'a handful',
 'a hang-up',
 'a heart-to-heart',
 'a heel',
 'a high-flyer',
 'a kickback',
 'à la',
 'a level',
 'a little',
 'a lot',
 'a major',
 'a mere',
 'a must',
 'a no-brainer',
 'a norange',
 'a one',
 'a pair',
 'a peach',
 'a pleasure',
 'a plus',
 'a posteriori',
 'a priori',
 'a propos',
 'a put-down',
 'a rip-off',
 'a roll',
 'a sec',
 'a shame',
 'a single',
 'a smokescreen',
 'a sport',
 'a stretch',
 'a student',
 'a to-do',
 'a ton',
 'a

In [27]:
helper = Fetcher(this_corpus.corpus_conn)

In [28]:
helper.create_bigram_query(bigrams)

In [29]:
helper.allocate_filtered_db()

Token Frequency

In [30]:
helper.conn.execute("""
    CREATE OR REPLACE TABLE token_freq AS
    SELECT 
        ug_1,
        ug_2,
        total_freq AS token_freq
    FROM filtered_db_total
    WHERE ug_1 IN (SELECT ug_1 FROM this_query) AND ug_2 IN (SELECT ug_2 FROM this_query)
""")

Type Frequencies

In [31]:
helper("""
    CREATE OR REPLACE TABLE type_1 AS
    SELECT 
        ug_2,
        COUNT( * ) AS typef_1
    FROM filtered_db_total
    WHERE ug_2 IN (SELECT ug_2 FROM this_query)
    GROUP BY
        ug_2
""")
helper("""
    CREATE OR REPLACE TABLE type_2 AS
    SELECT 
        ug_1,
        COUNT( * ) AS typef_2
    FROM filtered_db_total
    WHERE ug_1 IN (SELECT ug_1 FROM this_query)
    GROUP BY
        ug_1
""")

Dispersion

In [32]:
helper("""
CREATE OR REPLACE TABLE corpus_ngram_combs AS 
    SELECT corpus, ug_1, ug_2
    FROM (SELECT DISTINCT ug_1, ug_2 FROM filtered_db)
    CROSS JOIN (SELECT DISTINCT corpus FROM filtered_db)
""")

In [33]:
helper("""
    CREATE OR REPLACE TABLE disp_table AS
    SELECT corpus, ug_1, ug_2, COALESCE(freq, 0) as freq
    FROM corpus_ngram_combs
    LEFT JOIN filtered_db
    USING(corpus, ug_1, ug_2)
""")

In [34]:
helper(
    """
    CREATE OR REPLACE TABLE disp_table AS
    SELECT *
    FROM disp_table 
    LEFT JOIN corpus_proportions
    USING(corpus)
    WHERE ug_1 IN (SELECT ug_1 FROM this_query) AND ug_2 IN (SELECT ug_2 FROM this_query)
""")

In [35]:
helper(
    """
    CREATE OR REPLACE TABLE disp_table AS
    SELECT 
        corpus,
        disp_table.ug_1,
        disp_table.ug_2,
        freq / total_freq AS proportion,
        corpus_prop
    FROM disp_table
    LEFT JOIN filtered_db_total
    USING(ug_1, ug_2)
""")

In [36]:
helper("SELECT * FROM disp_table").fetch_df()

,corpus,ug_1,ug_2,proportion,corpus_prop
0,K,bishop,turned,0.0,0.119471
1,K,bored,by,0.0,0.119471
2,K,born,less,0.0,0.119471
3,K,both,mass,0.0,0.119471
4,K,breaking,waves,0.0,0.119471
...,...,...,...,...,...
3468345,E,this,miss,0.0,0.067143
3468346,B,financial,associations,0.0,0.067230
3468347,B,this,miss,0.0,0.067230
3468348,J,financial,associations,0.0,0.058538


In [37]:
helper(
    """
    WITH kld_table AS (SELECT 
        corpus,
        ug_1,
        ug_2,
        CASE
            WHEN proportion > 0 THEN proportion * log2(proportion / corpus_prop)
            ELSE 0
        END AS KLD
    FROM disp_table
    )
    SELECT 
        ug_1,
        ug_2,
        1 - pow(EXP(1), -SUM(KLD)) as dispersion
    FROM kld_table
    GROUP BY (ug_1, ug_2)
    ORDER BY dispersion
""").fetch_df()

,ug_1,ug_2,dispersion
0,on,the,0.001544
1,in,a,0.002235
2,for,the,0.002254
3,for,a,0.002643
4,at,the,0.002659
...,...,...,...
346830,european,goods,0.999958
346831,tone,it,0.999958
346832,belgian,treaty,0.999958
346833,any,bottle,0.999958


Associations @_@ Heavy use of CTE

In [38]:
helper(
    """
    CREATE OR REPLACE TABLE unigram_filtered AS
    SELECT 
        ug,
        SUM(freq) AS freq,
        SUM(SUM(freq)) OVER () AS total_ug,
    FROM unigram_db
    WHERE ug IN (SELECT ug_1 FROM this_query) OR ug IN (SELECT ug_2 FROM this_query)
    GROUP BY (ug)
""")

In [42]:
helper(
    """
    CREATE OR REPLACE TABLE rel_freqs AS
    SELECT 
        ug_1,
        ug_2,
        token_freq,
        unigram_filtered.freq AS ug_1_freq,
        unigram_filtered.total_ug 
    FROM token_freq
    LEFT JOIN unigram_filtered
    ON token_freq.ug_1 = unigram_filtered.ug
    """)
helper(
    """
    CREATE OR REPLACE TABLE rel_freqs AS
    SELECT 
        ug_1,
        ug_2,
        token_freq,
        ug_1_freq,
        unigram_filtered.freq AS ug_2_freq,
        rel_freqs.total_ug
    FROM rel_freqs
    LEFT JOIN unigram_filtered
    ON rel_freqs.ug_2 = unigram_filtered.ug
    """)

In [46]:
helper("""
    CREATE OR REPLACE TABLE assoc_table AS
    WITH probs_db AS (
        SELECT
            ug_1,
            ug_2, 
            token_freq / ug_1_freq AS prob_2_1,
            token_freq / ug_2_freq AS prob_1_2,
            ug_1_freq / total_ug AS prob_1,
            ug_2_freq / total_ug AS prob_2
        FROM rel_freqs
    ), all_probs AS (
        SELECT
            ug_1,
            ug_2,
            prob_2_1,
            prob_1_2,
            prob_1,
            prob_2,
            1 - prob_2_1 AS prob_no_2_1,
            1 - prob_1_2 AS prob_no_1_2,
            1 - prob_1 AS prob_no_1,
            1 - prob_2 AS prob_no_2
        FROM probs_db
    ), forward_kld AS (
        SELECT
            ug_1,
            ug_2,
            prob_2_1 * log2(prob_2_1 / prob_2) AS kld_1,
            CASE
                WHEN prob_no_2_1 = 0 THEN 0
                ELSE (1 - prob_2_1) * log2((1 - prob_2_1) / (1 - prob_2))
            END as kld_2
        FROM all_probs
    ), forward_assoc AS (
        SELECT
            ug_1,
            ug_2,
            1 - pow(EXP(1), -(kld_1 + kld_2)) AS fw_assoc
        FROM forward_kld
    ), backward_kld AS (
        SELECT
            ug_1,
            ug_2,
            prob_1_2 * log2(prob_1_2 / prob_1) AS kld_1,
            CASE
                WHEN prob_no_1_2 = 0 THEN 0
                ELSE (1 - prob_1_2) * log2((1 - prob_1_2) / (1 - prob_1))
            END as kld_2
        FROM all_probs
    ), backward_assoc AS (
        SELECT
            ug_1,
            ug_2,
            1 - pow(EXP(1), -(kld_1 + kld_2)) AS bw_assoc
        FROM backward_kld
    )
    SELECT *
    FROM forward_assoc
    LEFT JOIN backward_assoc
    USING(ug_1, ug_2)
    ORDER BY bw_assoc DESC
""")

In [48]:
helper("SELECT * FROM assoc_table").fetch_df()

,ug_1,ug_2,fw_assoc,bw_assoc
0,wishy,washy,1.000000e+00,1.000000e+00
1,garam,masala,1.000000e+00,1.000000e+00
2,nolle,prosequi,1.000000e+00,1.000000e+00
3,honoris,causa,1.000000e+00,1.000000e+00
4,trompe,l'oeil,1.000000e+00,1.000000e+00
...,...,...,...,...
346830,navy,with,4.499690e-11,1.578737e-13
346831,round,like,6.793455e-13,1.455502e-13
346832,fitted,by,2.432676e-11,1.443290e-13
346833,poverty,by,2.432676e-11,1.443290e-13
